In [1]:
import os
import cohere
import pandas as pd

co = cohere.Client(os.environ["COHERE_API_KEY"])

In [2]:
df = pd.read_csv("CSCO_data.csv")

In [3]:
df["text"] = df["date"].apply(lambda x: f"Stock price of CSCO on {x}")

In [4]:
df["label"] = df.apply(
    lambda row: "increase" if row["close"] > row["open"] else "decrease", axis=1
)

In [5]:
df.to_json("stocks.jsonl", orient="records", lines=True)

In [6]:
stock_price_dataset = co.datasets.create(
    name="multi-label-dataset",
    data=open("stocks.jsonl", "rb"),
    type="single-label-classification-finetune-input",
)

In [7]:
ds = co.wait(stock_price_dataset)
print(ds)

...
...
dataset=Dataset(id='multi-label-dataset-f5rrn1', name='multi-label-dataset', created_at=datetime.datetime(2024, 6, 5, 4, 13, 23, 714089, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2024, 6, 5, 4, 13, 23, 714089, tzinfo=datetime.timezone.utc), dataset_type='single-label-classification-finetune-input', validation_status='validated', validation_error=None, schema_='{"name":"cohere.single_label_classification_finetune_input","type":"record","fields":[{"name":"text","type":"string"},{"name":"label","type":"string"},{"name":"is_eval","type":"boolean","default":false}]}', required_fields=None, preserve_fields=None, dataset_parts=[DatasetPart(id='98ab001f-d2c8-484b-93c4-9edbd78b4e1e', name='stocks.avro', url='https://storage.googleapis.com/cohere-production-user-datasets/dataset-api-temp/30b5ef85-e023-4a52-8366-4e8ce5598652/5e56f1fd-3535-4eb7-a3cc-86143148f374/multi-label-dataset-f5rrn1/000_stocks.avro?X-Goog-Algorithm=GOOG4-RSA-SHA256&X-Goog-Credential=dataset%40cohere

In [8]:
from cohere.finetuning import FinetunedModel, Settings, BaseModel

In [9]:
finetune_model = co.finetuning.create_finetuned_model(
    request=FinetunedModel(
        name="stock-price",
        settings=Settings(
            base_model=BaseModel(
                base_type="BASE_TYPE_CLASSIFICATION",
            ),
            dataset_id=stock_price_dataset.id,
        ),
    )
)

BadRequestError: status_code: 400, body: {'message': 'finetuned_model.name already exists'}

In [10]:
print(
    f"fine-tune ID: {finetune_model.finetuned_model.id}, fine-tune status: {finetune_model.finetuned_model.status}"
)

NameError: name 'finetune_model' is not defined

In [11]:
ft = co.finetuning.get_finetuned_model(finetune_model.finetuned_model.id)

NameError: name 'finetune_model' is not defined

In [17]:
prompt = f"""
You have been provided data for stock price breakup for CSCO company.Your task is to return output as answer.

** Instructions ** 
1. Understand question and check if it is related to the stock of CSCO.
2. If question is not related to CSCO then response as "I don't have enough information about that."
3. If question is related to CSCO then reply answer in concise and proper manner.

** Example output **
[
    {{
        "question": "What is open price for CSCO on 13th Feb 2017",
        "answer": "Open Price for CSCO on 13th Feb 2020 is 31.67" 
    }},
    {{
        "question": "What was close price for CSCO on 13th Feb 2017",
        "answer": "Close Price for CSCO on 13th Feb 2020 is 31.97" 
    }},
    {{
        "question": "What was volume for CSCO on 13th Feb 2017",
        "answer": "Volume for CSCO on 13th Feb 2020 is 26518836" 
    }},
    {{
        "question": "on which date price was highest for CSCO?",
        "answer": "find highest value from csv file" 
    }},
    {{
        "question": "on which date high price was 31.38",
        "answer": "On 07th Feb 2017 high price was 31.38" 
    }},
    {{
        "question": "Who is PM of india?",
        "answer": "I don't have enough information about that." 
    }},
    {{
        "question": "What is 2?",
        "answer": "I don't have enough information about that." 
    }},
    {{
        "question": "Who won T20 world cup?",
        "answer": "I don't have enough information about that." 
    }},
    {{
        "question": "Who invented the computer?",
        "answer": "I don't have enough information about that." 
    }},
]

Please provide the answer in concise and proper manner.
"""

In [13]:
response = co.classify(
    model="dd96807a-9798-492f-afaf-75b4c69a73e2-ft", inputs=["Who is PM of India?"]
)

In [14]:
print("The confidence levels of the labels are: {}".format(response.classifications))

The confidence levels of the labels are: [ClassifyResponseClassificationsItem(id='cd39637f-f543-4888-b5a2-b35590fac0fd', input='who is pm of india?', prediction='increase', predictions=['increase'], confidence=0.6041654, confidences=[0.6041654], labels={'decrease': ClassifyResponseClassificationsItemLabelsValue(confidence=0.3958346), 'increase': ClassifyResponseClassificationsItemLabelsValue(confidence=0.6041654)}, classification_type='single-label')]


In [18]:
chat = input("Enter your question: ")

In [19]:
co.chat(
    model="dd96807a-9798-492f-afaf-75b4c69a73e2-ft",
    message=chat,
    return_prompt=True,
)

ApiError: status_code: 400, body: {'message': 'finetuned model with name dd96807a-9798-492f-afaf-75b4c69a73e2-ft is not ready for serving'}

Stock Price Chat Fine Tune